<a href="https://colab.research.google.com/github/fernandofsilva/thesis/blob/main/Notebook_Options_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup 

In [ ]:
import pandas as pd
import numpy as np
import math

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style('darkgrid')

# Load data

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Datasets/data_option.csv')
data.head()

,date,option,value,X,expire,option_type,T,S,volume,r,sigma
0,2015-01-05,PETRC53,0.65,9.61,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444
1,2015-01-05,PETRC50,0.90,9.01,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444
2,2015-01-05,PETRC51,0.81,9.21,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444
3,2015-01-05,PETRC52,0.73,9.41,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444
4,2015-01-05,PETRC54,0.61,9.81,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444


# Black-Scholes

In [ ]:
from scipy.stats import norm

# Auxiliary function for d_one risk-adjusted probability
def d11(S, X, T, r, sigma):
    """This is an auxiliary function and is not intended to be called externally."""
    return (np.log(S/X) + (r + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))

# Auxiliary function for d_two risk-adjusted probability
def d21(d1, T, sigma):
    """This is an auxiliary function and is not intended to be called externally."""
    return d1 - sigma * np.sqrt(T)

# Definition of the Black-Scholes delta function
def bs_delta(S, X, T, r, sigma, option_type):
    """Compute the delta of the Black-Scholes option pricing formula.

    Arguments:
    S           -- the current spot price of the underlying stock
    X           -- the option strike price
    T           -- the time until maturity (in fractions of a year)
    r           -- the risk-free interest rate
    sigma       -- the returns volatility of the underlying stock
    option_type -- the option type, either 'call' or 'put'

    Returns: a numpy.float_ representing the delta value
    Exceptions raised: ValueError if option_type is not 'call' or 'put'
    """
    if option_type == 'call':
        return norm.cdf(d11(S, X, T, r, sigma))
    elif option_type == 'put':
        return norm.cdf(-d11(S, X, T, r, sigma))
    else:
        # Raise an error if the option_type is neither a call nor a put
        raise ValueError("Option type is either 'call' or 'put'.")

# Definition of the Black-Scholes European option pricing formula
def black_scholes(S, X, T, r, sigma, option_type):
    """Price a European option using the Black-Scholes option pricing formula.

    Arguments:
    S           -- the current spot price of the underlying stock
    X           -- the option strike price
    T           -- the time until maturity (in fractions of a year)
    r           -- the risk-free interest rate
    sigma       -- the returns volatility of the underlying stock
    option_type -- the option type, either 'call' or 'put'

    Returns: a numpy.float_ representing the option value
    Exceptions raised: ValueError if option_type is not 'call' or 'put'
    """
    d_one = d11(S, X, T, r, sigma)
    d_two = d21(d_one, T, sigma)
    if option_type == 'call':
        return S * norm.cdf(d_one) - np.exp(-r * T) * X * norm.cdf(d_two)
    elif option_type == 'put':
        return -(S * norm.cdf(-d_one) - np.exp(-r * T) * X * norm.cdf(-d_two))
    else:
        # Raise an error if the option_type is neither a call nor a put
        raise ValueError("Option type is either 'call' or 'put'.")

def bs_pandas(row):
    value = black_scholes(
        row['S'], row['X'], row['T'], row['r']/100, row['sigma']/100, row['option_type']
    )
    return round(value, 2)

def delta_pandas(row):
    value = bs_delta(
        row['S'], row['X'], row['T'], row['r']/100, row['sigma']/100, row['option_type']
    )
    return int(round(value * 100, 0))

data['bs'] = data.apply(bs_pandas, axis=1)
data['delta'] = data.apply(delta_pandas, axis=1)

data.head()

,date,option,value,X,expire,option_type,T,S,volume,r,sigma,bs,delta
0,2015-01-05,PETRC53,0.65,9.61,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444,0.85,46
1,2015-01-05,PETRC50,0.90,9.01,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444,1.07,54
2,2015-01-05,PETRC51,0.81,9.21,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444,0.99,51
3,2015-01-05,PETRC52,0.73,9.41,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444,0.92,49
4,2015-01-05,PETRC54,0.61,9.81,2015-03-16,call,0.190476,8.61,674783884.0,11.57,77.182444,0.79,44


## Metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mse = mean_squared_error(data['value'], data['bs'])
mae = mean_absolute_error(data['value'], data['bs'])
mape = mean_absolute_percentage_error(data['value'], data['bs'])

print(f'Erro médio {mse:.2f}')
print(f'Raís do erro médio {np.sqrt(mse):.2f}')
print(f'Erro médio absoluto {mae:.2f}')
print(f'Percentual do erro médio absoluto {mape:.2f}')

Erro médio 0.10
Raís do erro médio 0.32
Erro médio absoluto 0.15
Percentual do erro médio absoluto 40.86


In [ ]:
data[data['option_type'] == 'put']

,date,option,value,X,expire,option_type,T,S,volume,r,sigma,bs,delta
17,2015-01-05,PETRO51,1.14,9.21,2015-03-16,put,0.190476,8.61,6.747839e+08,11.57,77.182444,1.39,49
18,2015-01-05,PETRN65,6.80,15.66,2015-02-09,put,0.099206,8.61,6.747839e+08,11.57,77.182444,6.88,99
19,2015-01-05,PETRN67,0.96,9.11,2015-02-09,put,0.099206,8.61,6.747839e+08,11.57,77.182444,1.06,53
20,2015-01-05,PETRN68,0.85,8.91,2015-02-09,put,0.099206,8.61,6.747839e+08,11.57,77.182444,0.95,49
21,2015-01-05,PETRN70,2.10,10.66,2015-02-09,put,0.099206,8.61,6.747839e+08,11.57,77.182444,2.19,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234495,2019-12-13,PETRX265,0.01,26.06,2019-12-16,put,0.003968,29.98,3.172882e+09,4.90,28.228712,0.00,0
234497,2019-12-13,PETRX347,4.35,34.31,2019-12-16,put,0.003968,29.98,3.172882e+09,4.90,28.228712,4.32,100
234498,2019-12-13,PETRX311,0.84,30.81,2019-12-16,put,0.003968,29.98,3.172882e+09,4.90,28.228712,0.84,94
234500,2019-12-13,PETRX286,0.02,28.31,2019-12-16,put,0.003968,29.98,3.172882e+09,4.90,28.228712,0.00,0


In [ ]:
bs_delta(8.61, 10.66, 0.099206, 11.57/100, 77.182444/100, 'put')

0.7610775316957249

In [ ]:
norm.cdf(-d11(8.61, 10.66, 0.099206, 11.57/100, 77.182444/100))

0.7610775316957249